## Tensorflow convert to Tflite

In [8]:
import tensorflow as tf
from tensorflow import keras
# (Optional) Plot actual vs. predicted using Plotly, as before
import plotly.graph_objs as go
print(tf.__version__)

2.19.0


In [9]:
import sys
sys.path.append('../scripts')
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import helper_functions
from scipy.io import arff


# Load and preprocess data as you did originally
# df = primer.ArffReader.read_file('../data/long_train_track_orginal/long_train_track_with_compass_and_light.arff')
data, meta = arff.loadarff('../data/long_train_track_orginal/long_train_track_with_compass_and_light.arff')
df = pd.DataFrame(data)
df['Compass'] = pd.to_numeric(df['Compass'], errors='coerce')
df['Light'] = pd.to_numeric(df['Light'], errors='coerce')
# Apply your custom feature engineering
timestamps = np.arange(len(df['Compass'])) * 0.1
df['turn_speed'] = helper_functions.compute_turn_speed(df['Compass'], timestamps=timestamps)
df['turn_speed'] = helper_functions.moving_average(df['turn_speed'], window_size=100)
df['turn_speed-10'] = df['turn_speed'].shift(-10)  # target 10 steps ahead

# Drop NaNs from moving average and shifting
df = df.dropna().reset_index(drop=True)

# Feature matrix (inputs) and target
features = df[['turn_speed']].to_numpy()
target = df['turn_speed-10'].to_numpy()


In [10]:
split_idx = int(0.2 * len(features))
X_train, X_test = features[:split_idx], features[split_idx:]
y_train, y_test = target[:split_idx], target[split_idx:]

scaler_x = MinMaxScaler()
X_train_scaled = scaler_x.fit_transform(X_train)
X_test_scaled = scaler_x.transform(X_test)

scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()


In [11]:
def create_tf_sequence_dataset(X, y, seq_length):
    Xs, ys = [], []
    for i in range(len(X) - seq_length):
        Xs.append(X[i:i+seq_length])
        ys.append(y[i+seq_length])
    return np.array(Xs), np.array(ys)

seq_length = 10

X_train_seq, y_train_seq = create_tf_sequence_dataset(X_train_scaled, y_train_scaled, seq_length)
X_test_seq, y_test_seq = create_tf_sequence_dataset(X_test_scaled, y_test_scaled, seq_length)


In [13]:
model = keras.Sequential([
    keras.layers.LSTM(12, input_shape=(seq_length, X_train_seq.shape[2]),unroll=True),
    keras.layers.Dense(1)
])

model.compile(optimizer=keras.optimizers.Adam(0.01), loss='mse')

model.fit(
    X_train_seq,
    y_train_seq,
    epochs=10,
    batch_size=10,
    validation_data=(X_test_seq, y_test_seq),
    verbose = 1
)

Epoch 1/10


/Users/nandu/.pyenv/versions/synth_env/versions/tf-env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0580 - val_loss: 0.0572
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0345 - val_loss: 0.0520
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0320 - val_loss: 0.0603
Epoch 4/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0299 - val_loss: 0.0383
Epoch 5/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0286 - val_loss: 0.0530
Epoch 6/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0269 - val_loss: 0.0408
Epoch 7/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0219 - val_loss: 0.0461
Epoch 8/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0176 - val_loss: 0.0378
Epoch 9/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0215 - val_loss: 0.0333
Epoch 10/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0144 - val_loss: 0.0499


In [14]:
# Inference on the test set
y_pred_seq = model.predict(X_test_seq).flatten()

# Inverse scaling for interpretability
y_pred_original = scaler_y.inverse_transform(y_pred_seq.reshape(-1, 1)).flatten()
y_test_original = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()



fig = go.Figure()
fig.add_trace(go.Scatter(y=y_test_original, mode='lines', name='Actual', line=dict(color='blue')))
fig.add_trace(go.Scatter(y=y_pred_original, mode='lines', name='Predicted', line=dict(color='red')))
fig.update_layout(title='Actual vs Predicted Turn Speed (TFLite Workflow)',
                  xaxis_title='Sample Index', yaxis_title='Turn Speed')
fig.show()


311/311 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step


In [15]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 12)             │           672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,057 (8.04 KB)

 Trainable params: 685 (2.68 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,372 (5.36 KB)

In [16]:
#save your model in the SavedModel format
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: saved_model/1/assets


INFO:tensorflow:Assets written to: saved_model/1/assets
W0000 00:00:1753437516.188244 7431203 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1753437516.188256 7431203 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-07-25 11:58:36.188405: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: saved_model/1
2025-07-25 11:58:36.189141: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-07-25 11:58:36.189150: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: saved_model/1
2025-07-25 11:58:36.195824: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-07-25 11:58:36.221099: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: saved_model/1
2025-07-25 11:58:36.231577: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 43174 microseconds

In [3]:
# Load the model 

# Load the TFLite model as an interpreter
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

/Users/nandu/.pyenv/versions/synth_env/versions/tf-env/lib/python3.10/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)

Input details: [{'name': 'serving_default_inputs:0', 'index': 0, 'shape': array([ 1, 10,  1], dtype=int32), 'shape_signature': array([-1, 10,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall:0', 'index': 207, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [12]:
sample_input = np.array(X_test_seq[0:1]).astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], sample_input)


In [ ]:
# Run inference
# interpreter.invoke()


RuntimeError: tensorflow/lite/kernels/read_variable.cc:67 variable != nullptr was not true.Node number 11 (READ_VARIABLE) failed to invoke.